In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
import code

class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, children=(), op='', label='', parent=None):
        self.data = data
        self.grad = 0
        self.label = label

        self._parent = parent
        self._children = list(children)
        self._op = op

    def _eigen_to_parent_grad(self):
        parent_op = self._parent._op
        siblings = [child for child in self._parent if child != self]

        if parent_op == '*':
            result = math.prod([s.data for s in siblings])
            
            return result
        if parent_op == '+':
            # count how often self 
            return math.sum([1 if s==self else 0 for s in siblings]) + 1
            
    def backward(self):
        if not self._parent:
            # this is the root node, so it's gradient is 1:
            self.grad = 1.0
        else:
            # internal or leaf-node:
            self.grad = self._parent.grad * self._eigen_to_parent_grad()

        # visit all child nodes:
        for child in self._children:
            child.backward()

    def __rassign__(self, other):
        v = Value(other.data, (self, ), '=')
        other.parent = v
        return v
    
    def __add__(self, other):
        # other = other if isinstance(other, Value) else Value(other)
        v = Value(self.data + other.data, (self, other), '+')
        self.parent = v
        other.parent = v
        return v

    def __mul__(self, other):
        # other = other if isinstance(other, Value) else Value(other)
        v = Value(self.data * other.data, (self, other), '*')
        self.parent = v
        other.parent = v
        return v

    # def relu(self):
    #     v = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')
    #     self.parent = v
    #     return v

    # def __neg__(self): # -self
    #     return self * -1

    # def __radd__(self, other): # other + self
    #     return self + other

    # def __sub__(self, other): # self - other
    #     return self + (-other)

    # def __rsub__(self, other): # other - self
    #     return other + (-self)

    # def __rmul__(self, other): # other * self
    #     return self * other

    # def __truediv__(self, other): # self / other
    #     return self * other**-1

    # def __rtruediv__(self, other): # other / self
    #     return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

In [5]:
a = Value(2.0, label='a')
b = Value(-3.5, label='b')
c = Value(1.0, label='d')

L = a*b + c

L.backward()

print(f"L_grad={L.grad}, a_grad={a.grad}, b_grad={b.grad}, c_grad={c.grad}")


L_grad=1.0, a_grad=1.0, b_grad=1.0, c_grad=1.0
